# Cervical Cancer Screening - Kaggle Challenge

Recently, Intel partnered with MobileODT to challenge Kagglers to develop an algorithm which accurately identifies a woman’s cervix type based on images. Their motivation: doing so will prevent ineffectual treatments and allow healthcare providers to give proper referral for cases that require more advanced treatment.

The following notebook is my solution for the presented task.

In this competition, we had to develop algorithms to correctly classify cervix types based on cervical images. These different types of cervix in our data set are all considered normal (not cancerous), but since the transformation zones aren't always visible, some of the patients require further testing while some don't. This decision is very important for the healthcare provider and critical for the patient. Identifying the transformation zones is not an easy task for the healthcare providers, therefore, an algorithm-aided decision will significantly improve the quality and efficiency of cervical cancer screening for these patients. 


## Dataset

The training dataset comprises of 1481 images belonging to 3 different categories, with the following distribution:
    
    1. Type 1 - 252 images
    2. Type 2 - 780 images
    3. Type 3 - 449 images
    
The competition was held in two stages where we were provided 2 test datasets for reporting our results. After stage 1, the output classes of stage 1 test images were released, so as to give kagglers a chance to improve and fine tune their models. The number of images provided for testing ast 2 stages are:
    1. Stage 1 Test: 512 images
    2. Stage 2 Test (Final): 4018 images
    
The final loss and accuracy were to be reported by tagging 4018 images.

The dataset is available on Kaggle [here](https://www.kaggle.com/c/intel-mobileodt-cervical-cancer-screening/data)

In [1]:
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, ZeroPadding2D, MaxPooling2D
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras import backend as K
K.set_image_dim_ordering('th')
K.set_floatx('float32')

import pandas as pd
import numpy as np
np.random.seed(17)

Using TensorFlow backend.


## Preprocessing Data

Data preprocessing comprises of the following steps:
    1. Resizing all images to same size (32 x 32 x 3)
    2. Normalizing pixel values
    3. Applying image deformations (Random Scaling + Rotations) for regularization
    4. Storing data in a loadable numpy format
    
Below, you will find functions for implementing above mentioned tasks

In [3]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2

def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (32, 32), interpolation=cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    return [path, resized]

def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.transpose((0, 3, 1, 2))
    fdata = fdata.astype('float32')
    fdata = fdata / 255
    return fdata

## Classification Model

### 1. Load prepared dataset

In [ ]:
train_data = np.load('train.npy')
train_target = np.load('train_target.npy')

### 2. Apply image tranformations

In [ ]:
datagen = ImageDataGenerator(rotation_range=0.3, zoom_range=0.3)
datagen.fit(train_data)
return datagen

### 3. Create a Convolutional Neural Network 

Using a CNN was a default choice given we have to build an image classifier.

We shall be using: 
    1. Two 2D-Convolutional layers followed by Max Pooling layers
    2. ReLU activations
    3. Dropout between output of second convolutional block and input of fully connected layer
    4. Two fully connected layers for classification with dropout
    5. Hyperbolic Tan activation for FC-1 layer
    6. Softmax activation for FC-2 layer (Obvious choice, given a multiclass classification problem)
    7. Adamax optimizer - a variant of Adam based on the infinity norm.

In [5]:
model = Sequential()
model.add(Convolution2D(4, 3, 3, activation='relu', dim_ordering='th', input_shape=(3, 32, 32))) #use input_shape=(3, 64, 64)
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
model.add(Convolution2D(8, 3, 3, activation='relu', dim_ordering='th'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), dim_ordering='th'))
model.add(Dropout(0.2))

model.add(Flatten())
model.add(Dense(12, activation='tanh'))
model.add(Dropout(0.1))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer='adamax', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(4, (3, 3), activation="relu", input_shape=(3, 32, 32..., data_format="channels_first")`
  from ipykernel import kernelapp as app
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(strides=(2, 2), data_format="channels_first", pool_size=(2, 2))`
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), activation="relu", data_format="channels_first")`
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(strides=(2, 2), data_format="channels_first", pool_size=(2, 2))`


**The model architecture looks as shown below:**

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 30, 30)         112       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 15, 15)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 8, 13, 13)         296       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 6, 6)           0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 8, 6, 6)           0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 288)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 12)                3468      
__________

### 4. Train model

In [ ]:
x_train,x_val_train,y_train,y_val_train = train_test_split(train_data,train_target,test_size=0.4, random_state=17)

#fitting data
model.fit_generator(datagen.flow(x_train,y_train, batch_size=15, shuffle=True), nb_epoch=200, samples_per_epoch=len(x_train), verbose=1, validation_data=(x_val_train, y_val_train))

### 5. Predict on Test Data

In [ ]:
test_data = np.load('test1.npy')
test_id = np.load('test_id1.npy')
print("creating predictions")
predictions = model.predict_proba(test_data)
print("predictions made")

### 6. Writing output in Kaggle Submission format

In [ ]:
df = pd.DataFrame(predictions, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission.csv', index=False)
print("submission created")

## Results

The simple convolutional model implemented in this notebook was able to generate a **score of 0.96407**.

**This helped me achieve a rank of #110 on Kaggle leaderboard.**

#### Future considerations:
    1. I believe a higher score can be achieved by Transfer Learning. Fine tuning a pretrained model such as Inception-V3, VGG19, ResNet-50 can definitely boost the model accuracy.
    
    2. Many kagglers reported improved results by using R-CNN like approach i.e generating bounding boxes around regions of interest and generating probability predictions.
    
I would definitely like exploring these ideas in future implementations!